# Iris Dataset

* Employ Naive Bayes and other classification techniques
* Develop a conclusion from each technique and compare results among techniques 
* Discuss/Compare all analysis techniques used on the iris dataset over the term (assignments 3 & 5): 
    * which worked best, which worked the least and why?

## Import Libraries

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pandas_profiling import ProfileReport
from sklearn.datasets import load_iris
from statsmodels.tools.eval_measures import rmse
from sklearn.preprocessing import normalize
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.inspection import permutation_importance
# from sklearn.cluster import AffinityPropagation, MeanShift, SpectralClustering, OPTICS, Birch
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.gaussian_process import GaussianProcessClassifier

from sklearn.neighbors import NearestCentroid

In [ ]:
# Set Plotting Scale Params
plt.rcParams['figure.figsize'] = (16, 12)
sns.set_style('darkgrid')
%matplotlib inline

## Defining Classes/Functions

In [ ]:
### Defining Classes/Functions

def classification(method, x_dat, y_dat, **params): 
    
    #fit model
    mod = Pipeline([ ('classify', method(**params)) ])
    mod.fit(x_dat, y_dat)
    y_pred = mod.predict(x_dat)
    
    #print results
    print("Results for {}:".format(method.__name__))
    print(classification_report(y_dat, y_pred))
    print("Test Accuracy: {}%".format(round(mod.score(x_dat, y_dat)*100,2)))
    
    #print confusion matrix
    y_pred_rf = y_pred
    y_true_rf = y_dat
    cm = confusion_matrix(y_true_rf, y_pred_rf)
    f, ax = plt.subplots(figsize =(5,5))
    sns.heatmap(cm,annot = True,linewidths=0.5,linecolor="red",fmt = ".0f",ax=ax)
    plt.xlabel("y_pred_rf")
    plt.ylabel("y_true_rf")
    plt.title('Confusion Matrix')
    plt.show()
    
    # Calculate permutation feature importance
    # permutation feature importance - the decrease in a model score when a single feature value is randomly shuffled
    # thus the drop in the model score is indicative of how much the model depends on the feature
    # (n_jobs=-1 means using all processors)
    try:
        imp = permutation_importance(mod, x_dat, y_dat, n_jobs=-1)
        
        #Generate feature importance plot
        plt.figure(figsize=(12,8))
        importance_data = pd.DataFrame({'feature':x_dat.columns, 'importance':imp.importances_mean})
        sns.barplot(x='importance', y='feature', data=importance_data)
        plt.title('Permutation Feature Importance')
        plt.xlabel('Mean Decrease in F1 Score')
        plt.ylabel('')
        plt.show()
    
    except:
        print('No Feature Importance Available')
        
    

## Load Dataset, Explore and Display Features

In [ ]:
iris = load_iris()
iris_df = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                     columns= iris['feature_names'] + ['target'])
iris_df['target'] = iris_df['target'].replace([0,1,2],['setosa', 'versicolor', 'virginica'])

In [ ]:
iris_df

In [ ]:
iris_df.shape

In [ ]:
iris_df.info()

In [ ]:
iris_df['target'].describe()

In [ ]:
iris_df.describe()

In [ ]:
profile = ProfileReport(iris_df)
profile

In [ ]:
iris_df.corr()

### Split into feature and target data

In [ ]:
# Note these x and y values will be used in the first set of classification models
# The second set of classification models uses this y value but a modified x value
x_iris = iris_df.drop(["target"], axis=1)
y_iris = iris_df["target"]

## Sample Decision Tree - not sure if we need this?  check on colors, classes

In [ ]:
tree = DecisionTreeClassifier()
tree.fit(x_iris, y_iris )

fig = plt.figure(figsize=(120,50))
fig = plt.figure(figsize=(60,25))
out = plot_tree(tree,filled=True, feature_names = x_iris .columns, rounded=True, proportion=True,\
                class_names = ['setosa', 'versicolor', 'virginica'])
for o in out:
    arrow = o.arrow_patch
    if arrow is not None:
        arrow.set_edgecolor('black')
        arrow.set_linewidth(4)

In [ ]:
featurelist=[]
for i in x_iris.columns:
    featurelist.append(i)

In [ ]:
print(export_text(tree, feature_names = featurelist))

## Classification Models
#### Note: where applicable, random_state=42 in our models sets a seed so that results will be reproducible

### Decision Tree Classifier

In [ ]:
classification(DecisionTreeClassifier, x_iris, y_iris, random_state=42)

### C-Support Vector Classifier

In [ ]:
classification(SVC, x_iris, y_iris, random_state=42)

### K Neighbors Classifier

In [ ]:
classification(KNeighborsClassifier, x_iris, y_iris, n_neighbors=5)

### Logistic Regression Classifier

In [ ]:
classification(LogisticRegression, x_iris, y_iris, random_state=42) 

### Gaussian Naive Bayes Classifier

In [ ]:
classification(GaussianNB, x_iris, y_iris)

### Multinomial Naive Bayes Classifier

In [ ]:
classification(MultinomialNB, x_iris, y_iris)

### Bernoulli Naive Bayes Classifier

In [ ]:
classification(BernoulliNB, x_iris, y_iris)

### Random Forest Classifier

In [ ]:
# Note: random_state=42 sets a seed so the results are reproducible
classification(RandomForestClassifier, x_iris, y_iris, n_estimators=100, random_state=42)  

### Linear Discriminant Classifier

In [ ]:
classification(LinearDiscriminantAnalysis, x_iris, y_iris)

### Quadratic Discriminant Analysis

In [ ]:
classification(QuadraticDiscriminantAnalysis, x_iris, y_iris)

### Neural Network Multi-layer Perceptron Classifier

In [ ]:
classification(MLPClassifier, x_iris, y_iris, random_state=42)

### Radius Neighbors Classifier

In [ ]:
classification(RadiusNeighborsClassifier, x_iris, y_iris, n_jobs=50)

### Passive Aggressive Classifier

In [ ]:
classification(PassiveAggressiveClassifier, x_iris, y_iris, random_state=42)

### Gaussian Process Classifier

In [ ]:
classification(GaussianProcessClassifier, x_iris, y_iris, random_state=42)

### Voting Classifier

In [ ]:
clf1 = LogisticRegression(multi_class='multinomial', random_state=42)
clf2 = RandomForestClassifier(n_estimators=50, random_state=42)
clf3 = GaussianNB()
estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3)]

classification(VotingClassifier, x_iris, y_iris, n_jobs=50, estimators = estimators)

### Nearest Centroid Classifier

In [ ]:
classification(NearestCentroid, x_iris, y_iris)

## Other Analysis Techniques Employed Throughout the Term

* EDA
* Clustering Techniques (with and without PCA):
    * KMeans
    * Agglomerative
    * DBScan
    * Mean Shift

## Conclusions